In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna


from sklearn.model_selection import train_test_split
from sklearn import metrics, svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform


/Users/kirillsobolev/Documents/GitHub/Introduction-to-ML/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load the dataset

In [2]:
df = pd.read_csv("/Users/kirillsobolev/Documents/GitHub/Introduction-to-ML/exercise1/lin_reg_df_clean.csv")
df.head()

,Ram,OpSys,Weight,Price_euros,Company_Acer,Company_Apple,Company_Asus,Company_Dell,Company_HP,Company_Lenovo,...,Storage_1_Type_Flash,Storage_1_Type_HDD,Storage_1_Type_SSD,Storage_2_Type_,Storage_2_Type_HDD,Storage_2_Type_Hybrid,Storage_2_Type_SSD,Gpu_Manufacturer_AMD,Gpu_Manufacturer_Intel,Gpu_Manufacturer_Nvidia
0,8,0,1.37,1339.69,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
1,8,0,1.34,898.94,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
2,8,0,1.86,575.00,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,1,0
3,16,0,1.83,2537.45,0,1,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
4,8,0,1.37,1803.60,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0


Initialize scaler, scale y variable

In [3]:
# initalize scaler
scalerY = MinMaxScaler()

# name of target variable
target_variable = "Price_euros"

# save min/max -values of target value
# for bettermetrics later
min_y = df[target_variable].min()
max_y = df[target_variable].max()

# scale y-variable
num_vars = [target_variable]
df[num_vars] = scalerY.fit_transform(df[num_vars])


Split the dataset.

In [4]:
# if you  have more than one independent variables, list them all here
# leave out the target variable! (dependent variable)
X = df.drop([target_variable], axis=1)

# have only the target variable here (dependent variable)
y = df[target_variable]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Fit the model

In [5]:
# connect standard scaler for X-values
# you can also use MinMaxScaler() if performance is not good enough
model = make_pipeline(StandardScaler(), svm.SVR())
model.fit(X_train, y_train)

predictions = model.predict(X_test)

Metrics

In [6]:
# MAE - Mean average error
mae = round(metrics.mean_absolute_error(y_test, predictions), 3)
# MSE - Mean square error
mse = round(metrics.mean_squared_error(y_test, predictions), 3)
# RMSE - Root mean square error
rmse = round(np.sqrt(metrics.mean_squared_error(y_test, predictions)), 3)
# R-squared. 0 = the model descibes the dataset poorly
# 1 = model describes the dataset perfectly
r2 = round(metrics.r2_score(y_test, predictions), 3)

metrics_df = pd.DataFrame({
    'Metric': ['Mean Squared Error', 'Root Mean Squared Error', 'Mean Absolute Error', 'R-squared'],
    # Metrics from previous exercise 1 linear regression model
    'Basic Linear regression': [97327.03, 311.97, 239.35, 0.77]
})
# add metrics of SVR model
metrics_df['Basic SVR'] = [mse, rmse, mae, r2]
metrics_df

,Metric,Basic Linear regression,Basic SVR
0,Mean Squared Error,97327.03,0.009
1,Root Mean Squared Error,311.97,0.092
2,Mean Absolute Error,239.35,0.072
3,R-squared,0.77,0.757


We can see that basic linear regression performs slightly better in this case

<h1>Trying Optuna again.</h1>
Warning, this is quite heavy. And take ariund 15 minutes to run.

In [ ]:
def objective(trial):
    # Define hyperparameters to optimize
    C = trial.suggest_float('C', 0.1, 5.0)  # Regularization parameter
    epsilon = trial.suggest_float('epsilon', 0.01, 1.0)  # Epsilon parameter
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf'])  # Kernel type

    # Create and train model with hyperparameters
    model = svm.SVR(C=C, epsilon=epsilon, kernel=kernel)
    model.fit(X_train, y_train)

    # Evaluate model
    y_pred = model.predict(X_test)
    mse = metrics.mean_squared_error(y_test, y_pred)

    return mse

# Perform hyperparameter optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Get best hyperparameters
best_params = study.best_params
best_C = best_params['C']
best_epsilon = best_params['epsilon']
best_kernel = best_params['kernel']

# Train model with best hyperparameters
best_model = svm.SVR(C=best_C, epsilon=best_epsilon, kernel=best_kernel)
best_model.fit(X_train, y_train)

# Evaluate best model
y_pred_best = best_model.predict(X_test)
mse_best = metrics.mean_squared_error(y_test, y_pred_best)

print("Best hyperparameters:", best_params)
print("Best MSE:", mse_best)


In [8]:
predictions = best_model.predict(X_test)
# MAE - Mean average error
mae = round(metrics.mean_absolute_error(y_test, predictions), 3)
# MSE - Mean square error
mse = round(metrics.mean_squared_error(y_test, predictions), 3)
# RMSE - Root mean square error
rmse = round(np.sqrt(metrics.mean_squared_error(y_test, predictions)), 3)
# R-squared. 0 = the model descibes the dataset poorly
# 1 = model describes the dataset perfectly
r2 = round(metrics.r2_score(y_test, predictions), 3)

metrics_df['SVR Optuna'] = [mse, rmse, mae, r2]
metrics_df

,Metric,Basic Linear regression,Basic SVR,SVR Optuna
0,Mean Squared Error,97327.03,0.009,0.015
1,Root Mean Squared Error,311.97,0.092,0.122
2,Mean Absolute Error,239.35,0.072,0.086
3,R-squared,0.77,0.757,0.578


Finetuning with Optuna in this case is quite a failure.

In [9]:
model = make_pipeline(StandardScaler(), svm.LinearSVR())
model.fit(X_train, y_train)

/Users/kirillsobolev/Documents/GitHub/Introduction-to-ML/.venv/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/kirillsobolev/Documents/GitHub/Introduction-to-ML/.venv/lib/python3.10/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvr', LinearSVR())])

In [10]:
predictions = model.predict(X_test)
# MAE - Mean average error
mae = round(metrics.mean_absolute_error(y_test, predictions), 3)
# MSE - Mean square error
mse = round(metrics.mean_squared_error(y_test, predictions), 3)
# RMSE - Root mean square error
rmse = round(np.sqrt(metrics.mean_squared_error(y_test, predictions)), 3)
# R-squared. 0 = the model descibes the dataset poorly
# 1 = model describes the dataset perfectly
r2 = round(metrics.r2_score(y_test, predictions), 3)

metrics_df['Basic LinearSVR'] = [mse, rmse, mae, r2]
metrics_df

,Metric,Basic Linear regression,Basic SVR,SVR Optuna,Basic LinearSVR
0,Mean Squared Error,97327.03,0.009,0.015,0.011
1,Root Mean Squared Error,311.97,0.092,0.122,0.105
2,Mean Absolute Error,239.35,0.072,0.086,0.074
3,R-squared,0.77,0.757,0.578,0.684


<h1>GridSearch and Linear SVR</h1>
Try to finetune LinearSVR model using GridSearch (manually try out the hyperparameters)

In [ ]:
param_grid = {
    # Best C was spotted in space less than 1 and around 0.5
    'C': [0.1, 0.4, 0.5, 0.6, 0.75, 1],
    'epsilon': [0.02, 0.03, 0.05, 0.07, 0.1, 0.15],
    'max_iter': [1000, 1200, 1500]
}

model = make_pipeline(StandardScaler(), GridSearchCV(svm.SVR(), param_grid, refit=True, verbose=3))
model.fit(X_train, y_train)

In [12]:
model[1].best_params_

{'C': 0.5, 'epsilon': 0.02, 'max_iter': 1500}

In [13]:
predictions = model.predict(X_test)
# MAE - Mean average error
mae = round(metrics.mean_absolute_error(y_test, predictions), 3)
# MSE - Mean square error
mse = round(metrics.mean_squared_error(y_test, predictions), 3)
# RMSE - Root mean square error
rmse = round(np.sqrt(metrics.mean_squared_error(y_test, predictions)), 3)
# R-squared. 0 = the model descibes the dataset poorly
# 1 = model describes the dataset perfectly
r2 = round(metrics.r2_score(y_test, predictions), 3)

metrics_df['GridSearch LinearSVR'] = [mse, rmse, mae, r2]
metrics_df

,Metric,Basic Linear regression,Basic SVR,SVR Optuna,Basic LinearSVR,GridSearch LinearSVR
0,Mean Squared Error,97327.03,0.009,0.015,0.011,0.007
1,Root Mean Squared Error,311.97,0.092,0.122,0.105,0.087
2,Mean Absolute Error,239.35,0.072,0.086,0.074,0.058
3,R-squared,0.77,0.757,0.578,0.684,0.787


After a bit of manual finetuning model looks way better than it was. Every metric and overall accuracy (up to 9 procent points!) improved significantly.

<h1>NuSVR and RandomizedSearchCV</h1>

In contrast to GridSearchCV, not all parameter values are tried out, but rather a fixed number of parameter settings is sampled from the specified distributions. The number of parameter settings that are tried is given by n_iter.

In [14]:
param_dist = {
    'C': uniform(0.1, 10),  # Uniform distribution between 0.1 and 10
    'nu': uniform(0.1, 0.9),
    'kernel': ['linear', 'rbf'],
}

In [15]:
# Create the RandomizedSearchCV object
model = make_pipeline(StandardScaler(), RandomizedSearchCV(svm.NuSVR(), param_distributions=param_dist, n_iter=10, cv=5, verbose=1))

model.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomizedsearchcv',
                 RandomizedSearchCV(cv=5, estimator=NuSVR(),
                                    param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x1520cfb80>,
                                                         'kernel': ['linear',
                                                                    'rbf'],
                                                         'nu': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x1520cfe80>},
                                    verbose=1))])

In [16]:
model[1].best_params_

{'C': 1.9976734516947037, 'kernel': 'rbf', 'nu': 0.8482039334964717}

In [17]:
predictions = model.predict(X_test)
# MAE - Mean average error
mae = round(metrics.mean_absolute_error(y_test, predictions), 3)
# MSE - Mean square error
mse = round(metrics.mean_squared_error(y_test, predictions), 3)
# RMSE - Root mean square error
rmse = round(np.sqrt(metrics.mean_squared_error(y_test, predictions)), 3)
# R-squared. 0 = the model descibes the dataset poorly
# 1 = model describes the dataset perfectly
r2 = round(metrics.r2_score(y_test, predictions), 3)

metrics_df['RandomizedSearchCV NuSVR'] = [mse, rmse, mae, r2]
metrics_df

,Metric,Basic Linear regression,Basic SVR,SVR Optuna,Basic LinearSVR,GridSearch LinearSVR,RandomizedSearchCV NuSVR
0,Mean Squared Error,97327.03,0.009,0.015,0.011,0.007,0.008
1,Root Mean Squared Error,311.97,0.092,0.122,0.105,0.087,0.088
2,Mean Absolute Error,239.35,0.072,0.086,0.074,0.058,0.059
3,R-squared,0.77,0.757,0.578,0.684,0.787,0.779


Randomized search is quite similar to the GridSearch.

Quite funny that in the case of SVR fine tunning didn't bring anything special, just may be few percent point more to overall accuracy (while finetunning basic regression gave a lot). And optuna tuning kind of totally ruined the model. However, optimization of SVR is more difficult than simple regression model.
